In [2]:
import copy

import rasterio
import os

import numpy as np
import pandas as pd
from scipy.sparse import csr_matrix

In [3]:
# 自定义类，用于封装tif文件的操作
class TifFile:
    def __init__(self, file_path):
        self.file_path = file_path
        with rasterio.open(self.file_path) as src:
            self.data = src.read(1)  # 数值矩阵
            self.shape = self.data.shape  # 矩阵大小
            self.transform = src.transform  # 坐标变换参数
            self.crs = src.crs  # 坐标参考系统
            self.bounds = src.bounds  # 边界坐标
            self.width = src.width  # 宽度
            self.height = src.height  # 高度
            self.count = src.count  # 波段数


# 文件路径字典
file_paths = {
    "GLcBld_Height": "datasets/GLcBld_Height.tif",
    "glcent_CHM": "datasets/glcent_CHM.tif",
    "glcent_DSM": "datasets/glcent_DSM.tif",
    "KernelD_GLcB_new": "datasets/KernelD_GLcB_new.tif",
    "LAI": "datasets/LAI.tif",
    "GLcBuildings": "datasets/GLcBuildings2_Volume.tif",
}

# 动态创建TifFile实例并存储为全局变量
for variable_name, file_path in file_paths.items():
    tif_GLcBld_Height = TifFile('datasets/GLcBld_Height.tif')
    tif_glcent_CHM = TifFile('datasets/glcent_CHM.tif')
    tif_glcent_DSM = TifFile('datasets/glcent_DSM.tif')
    tif_KernelD_GLcB_new = TifFile('datasets/KernelD_GLcB_new.tif')
    tif_LAI = TifFile('datasets/LAI.tif')
    tif_CLcBld_Volume = TifFile('datasets/GLcBuildings2_Volume.tif')


# 数据信息了解

In [3]:
tif_GLcBld_Height.data

array([[65535, 65535, 65535, ..., 65535, 65535, 65535],
       [65535, 65535, 65535, ..., 65535, 65535, 65535],
       [65535, 65535, 65535, ..., 65535, 65535, 65535],
       ...,
       [65535, 65535, 65535, ..., 65535, 65535, 65535],
       [65535, 65535, 65535, ..., 65535, 65535, 65535],
       [65535, 65535, 65535, ..., 65535, 65535, 65535]], dtype=uint16)

In [4]:
tif_CLcBld_Volume.data

array([[1.79e+308, 1.79e+308, 1.79e+308, ..., 1.79e+308, 1.79e+308,
        1.79e+308],
       [1.79e+308, 1.79e+308, 1.79e+308, ..., 1.79e+308, 1.79e+308,
        1.79e+308],
       [1.79e+308, 1.79e+308, 1.79e+308, ..., 1.79e+308, 1.79e+308,
        1.79e+308],
       ...,
       [1.79e+308, 1.79e+308, 1.79e+308, ..., 1.79e+308, 1.79e+308,
        1.79e+308],
       [1.79e+308, 1.79e+308, 1.79e+308, ..., 1.79e+308, 1.79e+308,
        1.79e+308],
       [1.79e+308, 1.79e+308, 1.79e+308, ..., 1.79e+308, 1.79e+308,
        1.79e+308]])

In [5]:

# 计算每个唯一值的数量
unique_values, counts = np.unique(tif_CLcBld_Volume.data, return_counts=True)

# 根据计数进行排序，倒序排列
sorted_indices = np.argsort(counts)[::-1]

# 使用排序后的索引重新排序唯一值和计数
sorted_unique_values = unique_values[sorted_indices]
sorted_counts = counts[sorted_indices]

# 打印倒序的结果
for value, count in zip(sorted_unique_values, sorted_counts):
    print(f"Value: {value}, Count: {count}")

Value: 1.79e+308, Count: 176186
Value: 0.0, Count: 10
Value: 928527.484323412, Count: 1
Value: 2392.7598229062087, Count: 1
Value: 2318.131658438784, Count: 1
Value: 2340.495701923216, Count: 1
Value: 2343.575103831704, Count: 1
Value: 2347.826367820166, Count: 1
Value: 2353.548112046014, Count: 1
Value: 2354.5256864662047, Count: 1
Value: 2354.9696999449357, Count: 1
Value: 2364.457176799977, Count: 1
Value: 2366.0500337517083, Count: 1
Value: 2376.561980676646, Count: 1
Value: 2384.630338184513, Count: 1
Value: 2395.054363017829, Count: 1
Value: 2307.860575352961, Count: 1
Value: 2396.169689892315, Count: 1
Value: 2396.378400067073, Count: 1
Value: 2399.5614518127695, Count: 1
Value: 2404.695830837603, Count: 1
Value: 2410.3992268842576, Count: 1
Value: 2416.2989189446193, Count: 1
Value: 2433.913310737055, Count: 1
Value: 2442.0005557177624, Count: 1
Value: 2446.626538902459, Count: 1
Value: 2451.9278098587065, Count: 1
Value: 2451.9818236281753, Count: 1
Value: 2314.075716565306, C

In [4]:

# 计算每个唯一值的数量
unique_values, counts = np.unique(tif_GLcBld_Height.data, return_counts=True)

# 根据计数进行排序，倒序排列
sorted_indices = np.argsort(counts)[::-1]

# 使用排序后的索引重新排序唯一值和计数
sorted_unique_values = unique_values[sorted_indices]
sorted_counts = counts[sorted_indices]

# 打印倒序的结果
for value, count in zip(sorted_unique_values, sorted_counts):
    print(f"Value: {value}, Count: {count}")

Value: 65535, Count: 139282
Value: 109, Count: 1798
Value: 249, Count: 1338
Value: 294, Count: 1312
Value: 3, Count: 1113
Value: 205, Count: 822
Value: 342, Count: 698
Value: 96, Count: 584
Value: 340, Count: 552
Value: 154, Count: 534
Value: 315, Count: 528
Value: 195, Count: 510
Value: 232, Count: 497
Value: 299, Count: 482
Value: 268, Count: 479
Value: 233, Count: 473
Value: 170, Count: 437
Value: 104, Count: 433
Value: 110, Count: 421
Value: 215, Count: 394
Value: 280, Count: 380
Value: 75, Count: 376
Value: 269, Count: 374
Value: 175, Count: 374
Value: 210, Count: 370
Value: 243, Count: 357
Value: 67, Count: 346
Value: 40, Count: 345
Value: 140, Count: 343
Value: 30, Count: 340
Value: 333, Count: 339
Value: 157, Count: 335
Value: 98, Count: 335
Value: 139, Count: 331
Value: 270, Count: 321
Value: 242, Count: 321
Value: 145, Count: 319
Value: 106, Count: 319
Value: 39, Count: 317
Value: 63, Count: 315
Value: 180, Count: 313
Value: 16, Count: 308
Value: 244, Count: 305
Value: 21, Co

In [5]:
# 计算基本统计数据（此部分与前面代码相同）
mean_value = np.mean(tif_GLcBld_Height.data)
std_dev = np.std(tif_GLcBld_Height.data)
min_value = np.min(tif_GLcBld_Height.data)
max_value = np.max(tif_GLcBld_Height.data)

print("\nBasic Statistics:")
print(f"Mean: {mean_value}")
print(f"Standard Deviation: {std_dev}")
print(f"Min: {min_value}")
print(f"Max: {max_value}")


Basic Statistics:
Mean: 47696.188133956304
Standard Deviation: 29118.9257780749
Min: 1
Max: 65535


In [7]:
tif_GLcBld_Height.transform

Affine(5.0, 0.0, 257976.73599999957,
       0.0, -5.0, 666174.3099000007)

In [8]:
tif_CLcBld_Volume.transform

Affine(5.0, 0.0, 257985.0418999996,
       0.0, -5.0, 666100.7359999996)

In [7]:
tif_GLcBld_Height.crs

CRS.from_epsg(27700)

In [8]:
tif_GLcBld_Height.bounds

BoundingBox(left=257976.73599999957, bottom=663899.3099000007, right=260081.73599999957, top=666174.3099000007)

In [9]:
tif_CLcBld_Volume.bounds

BoundingBox(left=257985.0418999996, bottom=663940.7359999996, right=260045.0418999996, top=666100.7359999996)

In [9]:
tif_GLcBld_Height.width

421

In [10]:
tif_CLcBld_Volume.width

412

In [10]:
tif_GLcBld_Height.height

455

In [11]:
tif_CLcBld_Volume.height

432

In [11]:
tif_GLcBld_Height.count

1

In [12]:
tif_CLcBld_Volume.count

1

In [12]:
tif_glcent_CHM.data

array([[28.882736  , 28.408676  , 27.841417  , ..., 15.216122  ,
        14.058662  ,  0.08586502],
       [31.331669  , 30.637243  , 31.719954  , ..., 13.888069  ,
        12.631687  ,  0.07413864],
       [31.908613  , 31.804506  , 32.027122  , ..., 10.486404  ,
         9.954441  ,  0.0562439 ],
       ...,
       [ 6.266613  ,  6.3237047 ,  1.4739332 , ..., 17.83626   ,
        18.70651   , 17.208393  ],
       [ 4.7787457 ,  1.4792271 ,  0.1911993 , ..., 12.996392  ,
        14.958178  , 18.35652   ],
       [ 0.25867653,  0.15101147,  0.16328621, ...,  0.0835458 ,
         0.13466549, 12.975633  ]], dtype=float32)

In [13]:
tif_list = [tif_GLcBld_Height, tif_glcent_CHM, tif_glcent_DSM, tif_KernelD_GLcB_new, tif_LAI]

# 创建一个空的列表，用于存储每个tif文件的属性
data = []

# 遍历每个TifFile对象，提取属性并存储到data列表中
data = [
    {
        "Name": tif.file_path.split('/')[-1],  # 提取文件名
        "Data": tif.data,
        "Data.shape": tif.shape,
        "Transform": tif.transform,
        "CRS": tif.crs,
        "Bounds": tif.bounds,
        "Width": tif.width,
        "Height": tif.height,
        "Band Count": tif.count
    }
    for tif in tif_list
]

# 将列表转换为DataFrame
df = pd.DataFrame(data)


In [14]:
df

,Name,Data,Data.shape,Transform,CRS,Bounds,Width,Height,Band Count
0,GLcBld_Height.tif,"[[65535, 65535, 65535, 65535, 65535, 65535, 65...","(455, 421)","(5.0, 0.0, 257976.73599999957, 0.0, -5.0, 6661...",(init),"(257976.73599999957, 663899.3099000007, 260081...",421,455,1
1,glcent_CHM.tif,"[[28.882736, 28.408676, 27.841417, 2.758522, 0...","(400, 400)","(5.0, 0.0, 258000.0, 0.0, -5.0, 666000.0, 0.0,...",(init),"(258000.0, 664000.0, 260000.0, 666000.0)",400,400,1
2,glcent_DSM.tif,"[[55.24, 54.737, 54.116, 28.998, 26.305, 36.85...","(400, 400)","(5.0, 0.0, 258000.0, 0.0, -5.0, 666000.0, 0.0,...",(init),"(258000.0, 664000.0, 260000.0, 666000.0)",400,400,1
3,KernelD_GLcB_new.tif,"[[244.3018, 275.4633, 306.76352, 337.6687, 367...","(245, 244)","(8.21323320000243, 0.0, 257999.8617497996, 0.0...",(init),"(257999.8617497996, 663991.944182601, 260003.8...",244,245,1
4,LAI.tif,"[[11.7030945, 11.513471, 11.286567, 1.2534088,...","(400, 400)","(5.0, 0.0, 258000.0, 0.0, -5.0, 666000.0, 0.0,...",(init),"(258000.0, 664000.0, 260000.0, 666000.0)",400,400,1


In [23]:
A = tif_glcent_CHM
B = tif_GLcBld_Height
C = tif_CLcBld_Volume
transform_A = A.transform
transform_B = B.transform
transform_C = C.transform
width_A = A.width
height_A = A.height
width_B = B.width
height_B = B.height
width_C = C.width
height_C = C.height

In [24]:
transform_A

Affine(5.0, 0.0, 258000.0,
       0.0, -5.0, 666000.0)

In [25]:
transform_B

Affine(5.0, 0.0, 257976.73599999957,
       0.0, -5.0, 666174.3099000007)

In [26]:
transform_C

Affine(5.0, 0.0, 257985.0418999996,
       0.0, -5.0, 666100.7359999996)

In [22]:
width_A

400

边界计算：
已知信息
A 的仿射变换: Affine(5.0, 0.0, 258000.0, 0.0, -5.0, 666000.0)
B 的仿射变换: Affine(5.0, 0.0, 257976.73599999997, 0.0, -5.0, 666174.3099999997)

A 的大小: width_A = 400，height_A = 400
B 的大小: width_B = 421，height_B = 455

计算边界（Bounding Box）
我们需要计算两个图像的空间范围（Bounding Box），即确定它们的左上角和右下角的地理坐标。

1. A 的边界
左上角（xmin_A, ymax_A）:
xmin_A = 258000.0
ymax_A = 666000.0
右下角（xmax_A, ymin_A）:
xmax_A = 258000.0 + 400 * 5.0 = 260000.0
ymin_A = 666000.0 + 400 * -5.0 = 664000.0
因此，A 的边界是：

(xmin_A, ymin_A, xmax_A, ymax_A) = (258000.0, 664000.0, 260000.0, 666000.0)
2. B 的边界
左上角（xmin_B, ymax_B）:
xmin_B = 257976.736
ymax_B = 666174.31
右下角（xmax_B, ymin_B）:
xmax_B = 257976.736 + 421 * 5.0 = 260081.736
ymin_B = 666174.31 + 455 * -5.0 = 663899.81
因此，B 的边界是：

(xmin_B, ymin_B, xmax_B, ymax_B) = (257976.736, 663899.81, 260081.736, 666174.31)

B完全覆盖A

C也完全覆盖A

In [30]:
import rasterio
from rasterio.windows import from_bounds

# 读取 C 的文件
with rasterio.open('datasets/GLcBuildings2_Volume.tif') as src_C:
    # A 的边界
    xmin_A, ymin_A, xmax_A, ymax_A = 258000.0, 664000.0, 260000.0, 666000.0

    # 计算 C 中与 A 重叠的窗口
    window = from_bounds(xmin_A, ymin_A, xmax_A, ymax_A, transform=src_C.transform)

    # 获取窗口的行列偏移和大小
    col_off, row_off = window.col_off, window.row_off
    width, height = window.width, window.height

    # 打印裁剪范围相对于原始 C 图像的行列范围
    print(f"裁剪范围在原始 C 图像中的列偏移: {col_off}, 行偏移: {row_off}")
    print(f"裁剪范围的宽度: {width} 像素，高度: {height} 像素")
    print(f"裁剪范围结束的列号: {col_off + width}, 行号: {row_off + height}")

    # 读取窗口内的 C 的数据
    C_overlap_data = src_C.read(window=window)

    # 生成裁剪后的 C 数据
    profile = src_C.profile
    profile.update({
        "height": window.height,
        "width": window.width,
        "transform": rasterio.windows.transform(window, src_C.transform)
    })


裁剪范围在原始 C 图像中的列偏移: 2.9916200000734534, 行偏移: 20.147199999919394
裁剪范围的宽度: 400.0 像素，高度: 400.0 像素
裁剪范围结束的列号: 402.99162000007345, 行号: 420.1471999999194


In [20]:
C_overlap_data

array([[[  181, 65535, 65535, ..., 65535, 65535, 65535],
        [  181,   181,   181, ..., 65535, 65535, 65535],
        [  181,   181,   181, ..., 65535, 65535, 65535],
        ...,
        [65535, 65535, 65535, ..., 65535, 65535, 65535],
        [65535, 65535, 65535, ..., 65535, 65535, 65535],
        [65535, 65535, 65535, ..., 65535, 65535, 65535]]], dtype=uint16)

In [32]:
CHM_tif = copy.deepcopy(tif_glcent_CHM)
DSM_tif = copy.deepcopy(tif_glcent_DSM)
LAI_tif = copy.deepcopy(tif_LAI)
BH_tif = copy.deepcopy(tif_glcent_CHM)
BV_tif = copy.deepcopy(tif_CLcBld_Volume)

In [33]:
BH_tif

In [24]:
BH_tif.data

array([[28.882736  , 28.408676  , 27.841417  , ..., 15.216122  ,
        14.058662  ,  0.08586502],
       [31.331669  , 30.637243  , 31.719954  , ..., 13.888069  ,
        12.631687  ,  0.07413864],
       [31.908613  , 31.804506  , 32.027122  , ..., 10.486404  ,
         9.954441  ,  0.0562439 ],
       ...,
       [ 6.266613  ,  6.3237047 ,  1.4739332 , ..., 17.83626   ,
        18.70651   , 17.208393  ],
       [ 4.7787457 ,  1.4792271 ,  0.1911993 , ..., 12.996392  ,
        14.958178  , 18.35652   ],
       [ 0.25867653,  0.15101147,  0.16328621, ...,  0.0835458 ,
         0.13466549, 12.975633  ]], dtype=float32)

In [25]:
B_overlap_data.shape

(1, 400, 400)

In [26]:

BH_tif.data = C
BH_tif.shape = C.shape

In [27]:
BH_tif.data

array([[  181, 65535, 65535, ..., 65535, 65535, 65535],
       [  181,   181,   181, ..., 65535, 65535, 65535],
       [  181,   181,   181, ..., 65535, 65535, 65535],
       ...,
       [65535, 65535, 65535, ..., 65535, 65535, 65535],
       [65535, 65535, 65535, ..., 65535, 65535, 65535],
       [65535, 65535, 65535, ..., 65535, 65535, 65535]], dtype=uint16)

In [28]:
list_tif = [
    BH_tif,  # 建筑物高度
    CHM_tif,  # 树冠高度
    LAI_tif,  # 叶面积指数
    DSM_tif  # 地形数据   
]

data1 = []

# 遍历每个TifFile对象，提取属性并存储到data列表中
data1 = [
    {
        "Name": tif.file_path.split('/')[-1],  # 提取文件名
        "Data": tif.data,
        "Data.shape": tif.shape,
        "Transform": tif.transform,
        "CRS": tif.crs,
        "Bounds": tif.bounds,
        "Width": tif.width,
        "Height": tif.height,
        "Band Count": tif.count
    }
    for tif in list_tif
]

In [29]:

df = pd.DataFrame(data1)

In [30]:
df

,Name,Data,Data.shape,Transform,CRS,Bounds,Width,Height,Band Count
0,glcent_CHM.tif,"[[181, 65535, 65535, 65535, 65535, 65535, 6553...","(400, 400)","(5.0, 0.0, 258000.0, 0.0, -5.0, 666000.0, 0.0,...",(init),"(258000.0, 664000.0, 260000.0, 666000.0)",400,400,1
1,glcent_CHM.tif,"[[28.882736, 28.408676, 27.841417, 2.758522, 0...","(400, 400)","(5.0, 0.0, 258000.0, 0.0, -5.0, 666000.0, 0.0,...",(init),"(258000.0, 664000.0, 260000.0, 666000.0)",400,400,1
2,LAI.tif,"[[11.7030945, 11.513471, 11.286567, 1.2534088,...","(400, 400)","(5.0, 0.0, 258000.0, 0.0, -5.0, 666000.0, 0.0,...",(init),"(258000.0, 664000.0, 260000.0, 666000.0)",400,400,1
3,glcent_DSM.tif,"[[55.24, 54.737, 54.116, 28.998, 26.305, 36.85...","(400, 400)","(5.0, 0.0, 258000.0, 0.0, -5.0, 666000.0, 0.0,...",(init),"(258000.0, 664000.0, 260000.0, 666000.0)",400,400,1


In [31]:
list_tifs = [BH_tif.data, CHM_tif.data, LAI_tif.data, DSM_tif.data]

In [32]:
list_tifs

[array([[  181, 65535, 65535, ..., 65535, 65535, 65535],
        [  181,   181,   181, ..., 65535, 65535, 65535],
        [  181,   181,   181, ..., 65535, 65535, 65535],
        ...,
        [65535, 65535, 65535, ..., 65535, 65535, 65535],
        [65535, 65535, 65535, ..., 65535, 65535, 65535],
        [65535, 65535, 65535, ..., 65535, 65535, 65535]], dtype=uint16),
 array([[28.882736  , 28.408676  , 27.841417  , ..., 15.216122  ,
         14.058662  ,  0.08586502],
        [31.331669  , 30.637243  , 31.719954  , ..., 13.888069  ,
         12.631687  ,  0.07413864],
        [31.908613  , 31.804506  , 32.027122  , ..., 10.486404  ,
          9.954441  ,  0.0562439 ],
        ...,
        [ 6.266613  ,  6.3237047 ,  1.4739332 , ..., 17.83626   ,
         18.70651   , 17.208393  ],
        [ 4.7787457 ,  1.4792271 ,  0.1911993 , ..., 12.996392  ,
         14.958178  , 18.35652   ],
        [ 0.25867653,  0.15101147,  0.16328621, ...,  0.0835458 ,
          0.13466549, 12.975633  ]], dty

## 天气数据

In [33]:
import pandas as pd

clean_weather_data = pd.read_csv('datasets/clean_weather_data.csv')
clean_weather_data

,day,tempMin,tempMax,summary,desc,cloudCover,humidity,windSpeed,visibility
0,2015-01-01,7.53,12.23,Possible light rain until evening.,rain,0.80,0.89,14.69,4.43
1,2015-01-02,3.58,7.15,Possible light rain throughout the day.,rain,0.62,0.79,15.04,5.64
2,2015-01-03,-0.61,6.54,Clear throughout the day.,rain,0.31,0.84,4.48,6.20
3,2015-01-04,-0.63,7.59,Mostly cloudy throughout the day.,partly-cloudy-day,0.78,0.85,4.35,6.22
4,2015-01-05,6.51,10.43,Overcast throughout the day.,partly-cloudy-day,0.85,0.91,6.20,5.91
...,...,...,...,...,...,...,...,...,...
1790,2019-11-26,8.55,10.57,Mostly cloudy throughout the day.,rain,0.96,0.85,6.80,10.00
1791,2019-11-27,5.79,9.40,Mostly cloudy throughout the day.,rain,0.87,0.85,6.82,9.03
1792,2019-11-28,2.30,6.67,Mostly cloudy throughout the day.,partly-cloudy-day,0.40,0.79,5.15,10.00
1793,2019-11-29,1.08,6.14,Mostly cloudy throughout the day.,clear-day,0.18,0.76,4.85,9.11


In [34]:
clean_weather_data_onehot = pd.get_dummies(clean_weather_data, columns=['summary', 'desc'])

In [35]:
clean_weather_data_onehot

,day,tempMin,tempMax,cloudCover,humidity,windSpeed,visibility,summary_Clear throughout the day.,summary_Dangerously windy in the morning and afternoon.,summary_Dangerously windy in the morning.,...,summary_Windy in the evening.,summary_Windy in the morning and afternoon.,summary_Windy in the morning.,summary_Windy overnight.,summary_Windy starting in the afternoon.,desc_clear-day,desc_cloudy,desc_fog,desc_partly-cloudy-day,desc_rain
0,2015-01-01,7.53,12.23,0.80,0.89,14.69,4.43,False,False,False,...,False,False,False,False,False,False,False,False,False,True
1,2015-01-02,3.58,7.15,0.62,0.79,15.04,5.64,False,False,False,...,False,False,False,False,False,False,False,False,False,True
2,2015-01-03,-0.61,6.54,0.31,0.84,4.48,6.20,True,False,False,...,False,False,False,False,False,False,False,False,False,True
3,2015-01-04,-0.63,7.59,0.78,0.85,4.35,6.22,False,False,False,...,False,False,False,False,False,False,False,False,True,False
4,2015-01-05,6.51,10.43,0.85,0.91,6.20,5.91,False,False,False,...,False,False,False,False,False,False,False,False,True,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1790,2019-11-26,8.55,10.57,0.96,0.85,6.80,10.00,False,False,False,...,False,False,False,False,False,False,False,False,False,True
1791,2019-11-27,5.79,9.40,0.87,0.85,6.82,9.03,False,False,False,...,False,False,False,False,False,False,False,False,False,True
1792,2019-11-28,2.30,6.67,0.40,0.79,5.15,10.00,False,False,False,...,False,False,False,False,False,False,False,False,True,False
1793,2019-11-29,1.08,6.14,0.18,0.76,4.85,9.11,False,False,False,...,False,False,False,False,False,True,False,False,False,False


# 数据整合 Data Integration 

In [36]:
# 展开为一维向量
BH_tif_flat = BH_tif.data.flatten()  # 建筑物高度
CHM_tif_flat = CHM_tif.data.flatten()  # 树冠高度
LAI_tif_flat = LAI_tif.data.flatten()  # 叶面积指数
DSM_tif_flat = DSM_tif.data.flatten()  # 地形数据

# 将所有空间特征组合在一起，形成一个二维数组 (640000, 4)
spatial_features = np.column_stack((BH_tif_flat, CHM_tif_flat, LAI_tif_flat, DSM_tif_flat))


In [37]:
spatial_features

array([[1.81000000e+02, 2.88827362e+01, 1.17030945e+01, 5.52400017e+01],
       [6.55350000e+04, 2.84086761e+01, 1.15134706e+01, 5.47369995e+01],
       [6.55350000e+04, 2.78414173e+01, 1.12865667e+01, 5.41160011e+01],
       ...,
       [6.55350000e+04, 8.35458040e-02, 1.83418334e-01, 1.90300000e+00],
       [6.55350000e+04, 1.34665489e-01, 2.03866199e-01, 1.94400001e+00],
       [6.55350000e+04, 1.29756327e+01, 5.34025335e+00, 1.47709999e+01]],
      dtype=float32)

In [38]:
clean_weather_data_onehot.shape

(1795, 99)

In [39]:
clean_weather_data_onehot.dtypes

day                        object
tempMin                   float64
tempMax                   float64
cloudCover                float64
humidity                  float64
                           ...   
desc_clear-day               bool
desc_cloudy                  bool
desc_fog                     bool
desc_partly-cloudy-day       bool
desc_rain                    bool
Length: 99, dtype: object

In [40]:
# 1. 将 'day' 列转换为 datetime 类型
clean_weather_data_onehot['day'] = pd.to_datetime(clean_weather_data_onehot['day'])

# 储存一个没有循环这一特征的表
clean_weather_data_onehot_without_cycle = copy.deepcopy(clean_weather_data_onehot)
clean_weather_data_onehot_without_cycle.drop(columns=['day'], inplace=True)

# 2. 提取年、月、日、星期、一年中的第几天，并添加为新的列
# 按顺序将列插入到最前面
clean_weather_data_onehot.insert(0, 'year', clean_weather_data_onehot['day'].dt.year)
clean_weather_data_onehot.insert(1, 'month', clean_weather_data_onehot['day'].dt.month)
clean_weather_data_onehot.insert(2, 'day_of_month', clean_weather_data_onehot['day'].dt.day)
clean_weather_data_onehot.insert(3, 'day_of_week', clean_weather_data_onehot['day'].dt.weekday)
clean_weather_data_onehot.insert(4, 'day_of_year', clean_weather_data_onehot['day'].dt.dayofyear)

# 3. 删除原始的 'day' 列（如果不需要）
clean_weather_data_onehot.drop(columns=['day'], inplace=True)

# 4. 将 bool 类型数据转换为 0 和 1
bool_columns = clean_weather_data_onehot.select_dtypes(include=['bool']).columns
clean_weather_data_onehot[bool_columns] = clean_weather_data_onehot[bool_columns].astype(int)

In [41]:
# 结果检查
clean_weather_data_onehot

,year,month,day_of_month,day_of_week,day_of_year,tempMin,tempMax,cloudCover,humidity,windSpeed,...,summary_Windy in the evening.,summary_Windy in the morning and afternoon.,summary_Windy in the morning.,summary_Windy overnight.,summary_Windy starting in the afternoon.,desc_clear-day,desc_cloudy,desc_fog,desc_partly-cloudy-day,desc_rain
0,2015,1,1,3,1,7.53,12.23,0.80,0.89,14.69,...,0,0,0,0,0,0,0,0,0,1
1,2015,1,2,4,2,3.58,7.15,0.62,0.79,15.04,...,0,0,0,0,0,0,0,0,0,1
2,2015,1,3,5,3,-0.61,6.54,0.31,0.84,4.48,...,0,0,0,0,0,0,0,0,0,1
3,2015,1,4,6,4,-0.63,7.59,0.78,0.85,4.35,...,0,0,0,0,0,0,0,0,1,0
4,2015,1,5,0,5,6.51,10.43,0.85,0.91,6.20,...,0,0,0,0,0,0,0,0,1,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1790,2019,11,26,1,330,8.55,10.57,0.96,0.85,6.80,...,0,0,0,0,0,0,0,0,0,1
1791,2019,11,27,2,331,5.79,9.40,0.87,0.85,6.82,...,0,0,0,0,0,0,0,0,0,1
1792,2019,11,28,3,332,2.30,6.67,0.40,0.79,5.15,...,0,0,0,0,0,0,0,0,1,0
1793,2019,11,29,4,333,1.08,6.14,0.18,0.76,4.85,...,0,0,0,0,0,1,0,0,0,0


In [42]:
bool_columns_without_cycle = clean_weather_data_onehot_without_cycle.select_dtypes(include=['bool']).columns
clean_weather_data_onehot_without_cycle[bool_columns_without_cycle] = clean_weather_data_onehot_without_cycle[
    bool_columns].astype(int)

In [43]:
clean_weather_data_onehot_without_cycle

,tempMin,tempMax,cloudCover,humidity,windSpeed,visibility,summary_Clear throughout the day.,summary_Dangerously windy in the morning and afternoon.,summary_Dangerously windy in the morning.,summary_Dangerously windy starting in the afternoon.,...,summary_Windy in the evening.,summary_Windy in the morning and afternoon.,summary_Windy in the morning.,summary_Windy overnight.,summary_Windy starting in the afternoon.,desc_clear-day,desc_cloudy,desc_fog,desc_partly-cloudy-day,desc_rain
0,7.53,12.23,0.80,0.89,14.69,4.43,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1
1,3.58,7.15,0.62,0.79,15.04,5.64,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1
2,-0.61,6.54,0.31,0.84,4.48,6.20,1,0,0,0,...,0,0,0,0,0,0,0,0,0,1
3,-0.63,7.59,0.78,0.85,4.35,6.22,0,0,0,0,...,0,0,0,0,0,0,0,0,1,0
4,6.51,10.43,0.85,0.91,6.20,5.91,0,0,0,0,...,0,0,0,0,0,0,0,0,1,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1790,8.55,10.57,0.96,0.85,6.80,10.00,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1
1791,5.79,9.40,0.87,0.85,6.82,9.03,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1
1792,2.30,6.67,0.40,0.79,5.15,10.00,0,0,0,0,...,0,0,0,0,0,0,0,0,1,0
1793,1.08,6.14,0.18,0.76,4.85,9.11,0,0,0,0,...,0,0,0,0,0,1,0,0,0,0


In [44]:
clean_weather_data_onehot_without_cycle.dtypes

tempMin                   float64
tempMax                   float64
cloudCover                float64
humidity                  float64
windSpeed                 float64
                           ...   
desc_clear-day              int32
desc_cloudy                 int32
desc_fog                    int32
desc_partly-cloudy-day      int32
desc_rain                   int32
Length: 98, dtype: object

In [45]:
# # 将其转换为稀疏矩阵
# clean_weather_data_onehot_sparse = csr_matrix(clean_weather_data_onehot)

In [46]:
# clean_weather_data_onehot_sparse

<1795x103 sparse matrix of type '<class 'numpy.float64'>'
	with 22886 stored elements in Compressed Sparse Row format>

In [47]:
# # 假设时间序列数据有1796个时间点
# n_time_points = clean_weather_data_onehot.shape[0]

In [48]:
# # 复制空间特征（每个时间点都对应相同的空间特征）
# spatial_features_repeated = np.tile(spatial_features, (n_time_points, 1))  # 维度变为 (1796, 640000*4)

In [49]:
# # 加载时间序列数据
# time_series_data = clean_weather_data_onehot_sparse

In [50]:
# # 通过np.hstack将时间序列特征和展开的空间特征结合在一起
# combined_data = np.hstack((spatial_features_repeated, np.repeat(time_series_data, 400*400, axis=0)))

In [51]:
windSpeed = clean_weather_data_onehot['windSpeed']

In [52]:
# import pandas as pd


In [53]:
# X = pd.DataFrame({
#     'BH_tif_flat': BH_tif_flat,
#     'CHM_tif_flat': CHM_tif_flat,
#     'LAI_tif_flat': LAI_tif_flat,
#     'DSM_tif_flat': DSM_tif_flat
# })

In [54]:
# Y = windSpeed

In [55]:
# X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size=0.2, random_state=42)

In [56]:
# x1 = BH_tif.data
# x2 = CHM_tif.data
# x3 = LAI_tif.data
# x4 = DSM_tif.data
# 
# y = windSpeed

In [57]:
# # 将列表转换为 numpy 数组
# y = np.array(y)
# 
# # 通过 np.repeat 和 np.reshape 扩展为 [1795, 400, 400]
# y_expanded = np.repeat(y, 400*400).reshape(1795, 400, 400)

In [58]:
# print(y_expanded.shape) # 应输出 (1795, 400, 400)

(1795, 400, 400)


In [59]:
# x1_expanded = np.repeat(x1[np.newaxis, :, :], 1795, axis=0)
# x2_expanded = np.repeat(x2[np.newaxis, :, :], 1795, axis=0)
# x3_expanded = np.repeat(x3[np.newaxis, :, :], 1795, axis=0)
# x4_expanded = np.repeat(x4[np.newaxis, :, :], 1795, axis=0)
# x1_expanded.shape

(1795, 400, 400)

In [60]:
# x1_expanded

array([[[  181, 65535, 65535, ..., 65535, 65535, 65535],
        [  181,   181,   181, ..., 65535, 65535, 65535],
        [  181,   181,   181, ..., 65535, 65535, 65535],
        ...,
        [65535, 65535, 65535, ..., 65535, 65535, 65535],
        [65535, 65535, 65535, ..., 65535, 65535, 65535],
        [65535, 65535, 65535, ..., 65535, 65535, 65535]],

       [[  181, 65535, 65535, ..., 65535, 65535, 65535],
        [  181,   181,   181, ..., 65535, 65535, 65535],
        [  181,   181,   181, ..., 65535, 65535, 65535],
        ...,
        [65535, 65535, 65535, ..., 65535, 65535, 65535],
        [65535, 65535, 65535, ..., 65535, 65535, 65535],
        [65535, 65535, 65535, ..., 65535, 65535, 65535]],

       [[  181, 65535, 65535, ..., 65535, 65535, 65535],
        [  181,   181,   181, ..., 65535, 65535, 65535],
        [  181,   181,   181, ..., 65535, 65535, 65535],
        ...,
        [65535, 65535, 65535, ..., 65535, 65535, 65535],
        [65535, 65535, 65535, ..., 65535, 655

In [61]:
# # 构建滞后项 y_(t-1)
# y_lagged = np.roll(y_expanded, shift=1, axis=0)
# y_lagged[0, :, :] = 0 

In [62]:
# # 构建设计矩阵 X 和响应变量 Y
# X = np.stack([y_lagged, x1_expanded, x2_expanded, x3_expanded, x4_expanded], axis=-1)
# Y = y_expanded

In [63]:
# # 将数据重塑为 2D 以适应 statsmodels 的 OLS
# X_reshaped = X.reshape(-1, 5)
# Y_reshaped = Y.reshape(-1)

In [64]:
# import statsmodels.api as sm

In [65]:
# # 拟合模型
# X_reshaped = sm.add_constant(X_reshaped)  # 加入截距项
# model = sm.OLS(Y_reshaped, X_reshaped)
# results = model.fit()

KeyboardInterrupt: 

In [ ]:
# # 输出模型结果
# print(results.summary())

In [ ]:
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error

In [ ]:
# # 划分训练集和测试集
# X_train, X_test, Y_train, Y_test = train_test_split(X_reshaped, Y_reshaped, test_size=0.2, random_state=42)

In [ ]:
# # 构建随机森林模型
# rf_model = RandomForestRegressor(n_estimators=100, random_state=42)
# rf_model.fit(X_train, Y_train)

In [ ]:
# # 预测并评估模型
# Y_pred = rf_model.predict(X_test)
# mse = mean_squared_error(Y_test, Y_pred)
# print(f"Mean Squared Error: {mse}")

In [ ]:
# import matplotlib.pyplot as plt
# 
# # 提取特征重要性
# feature_importances = rf_model.feature_importances_
# features = ['y_lagged', 'x1', 'x2', 'x3', 'x4']
# 
# # 可视化特征重要性
# plt.barh(features, feature_importances)
# plt.xlabel("Feature Importance")
# plt.ylabel("Features")
# plt.title("Feature Importance in Random Forest Model")
# plt.show()

# 网格数据差值

In [ ]:
import numpy as np

# 已知点的坐标和温度值
points = np.array([
    [0, 0],  # 点 A
    [1, 0],  # 点 B
    [0, 1]   # 点 C
])

temperatures = np.array([5, 6, 7])  # 点 A, B, C 的温度值

# 半变异矩阵
Gamma = np.array([
    [0, 1, 1, 1],          # 点 A 到 A, B, C 和约束条件
    [1, 0, np.sqrt(2), 1], # 点 B 到 A, B, C 和约束条件
    [1, np.sqrt(2), 0, 1], # 点 C 到 A, B, C 和约束条件
    [1, 1, 1, 0]           # 约束条件
])

# 目标点 P 的坐标
P = np.array([0.5, 0.5])

# 计算 P 到 A, B, C 的半变异函数值
gamma = np.array([
    np.sqrt(np.sum((P - points[0])**2)),  # 到 A 的距离
    np.sqrt(np.sum((P - points[1])**2)),  # 到 B 的距离
    np.sqrt(np.sum((P - points[2])**2)),  # 到 C 的距离
    1  # 约束条件
])

# 解克里金系统
lambdas = np.linalg.solve(Gamma, gamma)

# 提取权重
lambda_A, lambda_B, lambda_C, mu = lambdas

# 使用权重计算插值点 P 的温度值
Z_P = lambda_A * temperatures[0] + lambda_B * temperatures[1] + lambda_C * temperatures[2]

print(f"点 P (0.5, 0.5) 的估计温度值为: {Z_P:.2f}")


In [ ]:
import pandas as pd

In [ ]:
# 输入6个点的坐标和天气数据值

# 提取的数据如下：
data_5_extra_points = [
    ['Renfrew Weather', (55.8739065, -4.3887910), (250756.73, 666880.47), '31/07/2024 09:30:00', 8.4, 17, 57, 0.4],
    ['Ralston Weather Station', (55.8466298, -4.3708027), (251848.22, 663740.00), '31/07/2024 09:30:00', 8.1, 11.8, 78.2, 0.9],
    ['Cathcart', (55.8114036, -4.2603826), (258548.22, 659684.28), '31/07/2024 09:28:07', 10, 15.9, 68, 0],
    ['Clincarthill', (55.8253972, -4.2119939), (261484.32, 661295.42), '31/07/2024 09:29:36', 9.3, 15.1, 68, 0.8],
    ['Cambuslang Scotland', (55.8253972, -4.2119939), (261484.32, 661295.42), '31/07/2024 09:31:36', 8.9, 13.4, 74, 0]
]

# 直接将数据列表转换为 Pandas DataFrame
columns = ['Location', 'A Coordinates (Lat, Lon)', 'B Coordinates (X_B, Y_B)', 'Report Date / Time', 'Dew Point (°C)', 'Air Temperature (°C)', 'Relative Humidity (%)', 'Wind Speed (kn)']
df_5_extra_points = pd.DataFrame(data, columns=columns)

# 显示 DataFrame
df_5_extra_points


In [ ]:
# 从DataFrame中提取B坐标系的点和相关数据
B_points = np.array([item[2] for item in data_5_extra_points])
dew_points = np.array(df_5_extra_points['Dew Point (°C)'])
air_temps = np.array(df_5_extra_points['Air Temperature (°C)'])
humidity = np.array(df_5_extra_points['Relative Humidity (%)'])
wind_speeds = np.array(df_5_extra_points['Wind Speed (kn)'])


In [ ]:
B_points

In [ ]:
dew_points

In [ ]:
air_temps

In [ ]:
humidity

In [ ]:
wind_speeds

In [ ]:
# 定义网格
x_min, x_max = 258000, 260000
y_min, y_max = 664000, 666000
grid_x, grid_y = np.mgrid[x_min:x_max:400j, y_min:y_max:400j]
grid_points = np.vstack([grid_x.ravel(), grid_y.ravel()]).T  # 是一个 (160000, 2) 的数组，保存了所有网格点的 (x, y) 坐标

In [ ]:
grid_points

In [ ]:
# 定义半变异矩阵Gamma，简化处理为欧式距离
def compute_gamma(points, P):
    dist = np.sqrt(np.sum((points - P)**2, axis=1))
    return np.hstack([dist, 1])


In [ ]:
# 函数：计算克里金插值
def kriging_interpolation(B_points, values, grid_points):
    # 构造Gamma矩阵
    n_points = len(B_points)
    Gamma = np.ones((n_points + 1, n_points + 1))
    for i in range(n_points):
        for j in range(n_points):
            Gamma[i, j] = np.sqrt(np.sum((B_points[i] - B_points[j])**2))

    # 添加一个小的正数扰动到对角线以避免奇异矩阵
    Gamma[np.diag_indices_from(Gamma)] += 1e-10

    Gamma[-1, -1] = 0

    # 插值结果存储
    interpolated_values = []

    for P in grid_points:
        gamma = compute_gamma(B_points, P)
        lambdas = np.linalg.solve(Gamma, gamma)
        interpolated_value = np.sum(lambdas[:-1] * values)
        interpolated_values.append(interpolated_value)

    return np.array(interpolated_values)


In [ ]:
# 对每个变量进行插值
dew_point_grid = kriging_interpolation(B_points, dew_points, grid_points)
air_temp_grid = kriging_interpolation(B_points, air_temps, grid_points)
humidity_grid = kriging_interpolation(B_points, humidity, grid_points)
wind_speed_grid = kriging_interpolation(B_points, wind_speeds, grid_points)


In [ ]:
# 将结果存储为DataFrame
grid_df = pd.DataFrame({
    'X': grid_points[:, 0],
    'Y': grid_points[:, 1],
    'Dew Point (°C)': dew_point_grid,
    'Air Temperature (°C)': air_temp_grid,
    'Relative Humidity (%)': humidity_grid,
    'Wind Speed (kn)': wind_speed_grid
})

In [ ]:
grid_df

In [ ]:
# 匹配地形数据与天气数据的形状

# 展平地形数据
X = np.array([tif.ravel() for tif in list_tifs]).T # (160000, 4)

In [ ]:
X

In [ ]:
# 提取天气数据
Y = grid_df[['Dew Point (°C)', 'Air Temperature (°C)', 'Relative Humidity (%)', 'Wind Speed (kn)']].values # (160000, 4)

In [61]:
Y

array([[ 8.99990701, 14.6003705 , 69.43959655,  0.40486841],
       [ 8.99925483, 14.600463  , 69.43653002,  0.40505876],
       [ 8.99860355, 14.60055833, 69.43345515,  0.40524862],
       ...,
       [ 8.86748666, 14.82650657, 67.81233419,  0.42656228],
       [ 8.86716808, 14.82715992, 67.80830767,  0.42660798],
       [ 8.86684982, 14.82781394, 67.80427948,  0.42665352]])

In [62]:
# # 构建存储结果的表格
# importance_matrix = np.zeros((4, 4))

In [66]:
# from sklearn.ensemble import RandomForestRegressor

In [67]:
# # 对每一个Y参数构建随机森林模型并计算重要性
# for i in range(4):
#     y = Y[:, i]  # 第 i 个 Y 参数
#     model = RandomForestRegressor(n_estimators=100, random_state=42)
#     model.fit(X, y)
#     importance_matrix[:, i] = model.feature_importances_

In [68]:
# # 将结果转化为DataFrame以便显示
# importance_df = pd.DataFrame(importance_matrix,
#                              columns=['Dew Point (°C)', 'Air Temperature (°C)', 'Relative Humidity (%)', 'Wind Speed (kn)'],
#                              index=['建筑物高度', '树冠高度', '叶面积指数', '地形数据'])


In [69]:
# # 显示结果
# importance_df

,Dew Point (°C),Air Temperature (°C),Relative Humidity (%),Wind Speed (kn)
建筑物高度,0.123744,0.140821,0.136987,0.112652
树冠高度,0.207721,0.210720,0.208571,0.190514
叶面积指数,0.207112,0.210738,0.209923,0.201741
地形数据,0.461423,0.437720,0.444520,0.495094


In [76]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.model_selection import cross_val_score
from skopt import BayesSearchCV
from skopt.space import Real, Categorical, Integer
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error, r2_score, mean_absolute_error, mean_absolute_percentage_error

from tqdm import tqdm

In [ ]:
# 读取数据
X 

In [ ]:
# 回调函数类，用于存储每次迭代的参数和评估指标，并更新tqdm进度条
class StoreResultsCallback:
    def __init__(self, total_iterations, X, Y, model_idx):
        self.parameters = []
        self.mse_scores = []
        self.r2_scores = []
        self.mae_scores = []
        self.rmse_scores = []
        self.mape_scores = []
        self.X = X
        self.Y = Y
        self.model_idx = model_idx
        self.pbar = tqdm(total=total_iterations, desc=f"BayesSearchCV Iterations for Model {model_idx+1}")

    def __call__(self, result):
        self.parameters.append(result.x)  # 储存当前参数
        y_pred = bayes_search.best_estimator_.predict(self.X)
        mse = mean_squared_error(self.Y[:, self.model_idx], y_pred)
        r2 = r2_score(self.Y[:, self.model_idx], y_pred)
        mae = mean_absolute_error(self.Y[:, self.model_idx], y_pred)
        rmse = np.sqrt(mse)
        mape = mean_absolute_percentage_error(self.Y[:, self.model_idx], y_pred)

        # 储存每次迭代的评估指标
        self.mse_scores.append(mse)
        self.r2_scores.append(r2)
        self.mae_scores.append(mae)
        self.rmse_scores.append(rmse)
        self.mape_scores.append(mape)

        # 更新进度条
        self.pbar.update(1)

    def __del__(self):
        # 在进度条完成后关闭它
        self.pbar.close()

In [72]:
# 定义随机森林的超参数搜索空间
search_space = {
    'n_estimators': Integer(100, 300),  # 决策树数量
    'max_depth': Integer(10, 30),  # 最大深度
    'max_features': Categorical(['auto', 'sqrt', 'log2']),  # 最大特征数
    'min_samples_split': Integer(2, 10),  # 最小分裂样本数
    'min_samples_leaf': Integer(1, 4),  # 最小叶子节点样本数
    'bootstrap': Categorical([True, False])  # 是否使用自助法
}

In [73]:
# 初始化随机森林模型
model = RandomForestRegressor(random_state=42, oob_score=True)

In [ ]:
# 用于存储每个模型的结果
all_results = []

In [ ]:
# 用于储存模型的特征重要性
importance_matrix = np.zeros((4, len(search_space)))

In [ ]:
n_iter = 10  # 迭代次数

In [74]:
# 使用贝叶斯优化进行超参数搜索
bayes_search = BayesSearchCV(
    estimator=model,
    search_spaces=search_space,
    n_iter=n_iter,  # 迭代次数，可以调整  表示贝叶斯优化将会搜索 n_iter 组不同的超参数组合。每一组超参数组合将会被评估一次。
    cv=5,  # 5 折交叉验证  对于每一个超参数组合，数据将会被分成 5 份，每次选择其中 4 份作为训练集，剩下的 1 份作为验证集，共进行 5 次训练和验证，最终得到该超参数组合的平均验证得分。
    
    # 结合n_iter和cv参数，BayesSearchCV将会在总共 10(n_iter) * 5 = 50 次训练和验证之后，返回最佳的超参数组合。
    # 每组超参数组合只会被评估一次，但在这一次评估中，它会经过 5 折交叉验证得到性能评分。
    # n_iter 次迭代对应于搜索和评估 n_iter 组不同的超参数组合，而不是每个超参数组合迭代 n_iter 次。
    # 即，10(n_iter)种不同的超参数组合，每种组合都会进行5折交叉验证，最终得到最佳的超参数组合。
    
    n_jobs=-1,  # 使用所有可用的核
    verbose=2,  # 显示详细信息
    scoring='neg_mean_squared_error'  # 使用负均方误差作为评分
)

In [ ]:
for i in range(4):  # 针对每个Y参数执行BayesSearchCV
    print(f"Optimizing model {i+1}/4")
    
    # 实例化回调函数，包含tqdm进度条
    store_results_callback = StoreResultsCallback(total_iterations=n_iter)

    # 开始贝叶斯优化，并传入回调函数
    bayes_search.fit(X, Y[:, i], callback=[store_results_callback])

    # 将结果存储
    all_results.append((store_results_callback, bayes_search.best_params_))
    
    # 获取特征重要性
    importance_matrix[:, i] = bayes_search.best_estimator_.feature_importances_


In [ ]:
# 打印最终的超参数优化和评估指标的具体值
for i, (result, best_params) in enumerate(all_results):
    print(f"\nModel {i+1}: Best Parameters: {best_params}")
    print(f"Final Evaluation Metrics for Model {i+1}:")
    print(f"  MSE: {result.mse_scores[-1]:.4f}")
    print(f"  R²: {result.r2_scores[-1]:.4f}")
    print(f"  MAE: {result.mae_scores[-1]:.4f}")
    print(f"  RMSE: {result.rmse_scores[-1]:.4f}")
    print(f"  MAPE: {result.mape_scores[-1]:.4f}")

In [ ]:
# 绘制每个参数在优化过程中的变化
def plot_param_evolution(all_results, param_name, param_index):
    plt.figure(figsize=(14, 8))
    for i, result in enumerate(all_results):
        param_values = [p[param_index] for p in result.parameters]
        plt.plot(range(len(param_values)), param_values, label=f'Model {i+1}')
    plt.title(f'{param_name} evolution during Bayes Optimization')
    plt.xlabel('Iteration')
    plt.ylabel(param_name)
    plt.legend()
    plt.grid(True)
    plt.show()

In [ ]:
# 绘制评估指标在优化过程中的变化
def plot_metric_evolution(all_results, metric_name, metric_values):
    plt.figure(figsize=(14, 8))
    for i, result in enumerate(all_results):
        plt.plot(range(len(metric_values[i])), metric_values[i], label=f'Model {i+1}')
    plt.title(f'{metric_name} evolution during Bayes Optimization')
    plt.xlabel('Iteration')
    plt.ylabel(metric_name)
    plt.legend()
    plt.grid(True)
    plt.show()

In [ ]:
# 参数索引对应到参数名称
param_names = ['n_estimators', 'max_depth', 'max_features', 'min_samples_split', 'min_samples_leaf', 'bootstrap']
param_indices = [0, 1, 2, 3, 4, 5]

# 绘制所有参数的变化
for param_name, param_index in zip(param_names, param_indices):
    plot_param_evolution(all_results, param_name, param_index)

In [ ]:
# 绘制评估指标的变化
metrics = ['MSE', 'R²', 'MAE', 'RMSE', 'MAPE']
metric_values = [
    [result.mse_scores for result in all_results],
    [result.r2_scores for result in all_results],
    [result.mae_scores for result in all_results],
    [result.rmse_scores for result in all_results],
    [result.mape_scores for result in all_results],
]

for metric_name, metric_value in zip(metrics, metric_values):
    plot_metric_evolution(all_results, metric_name, metric_value)

In [ ]:
# 将特征重要性结果转化为DataFrame
importance_df = pd.DataFrame(importance_matrix,
                             columns=['Dew Point (°C)', 'Air Temperature (°C)', 'Relative Humidity (%)', 'Wind Speed (kn)'],
                             index=['建筑物高度', '树冠高度', '叶面积指数', '地形数据'])

print("Feature Importances:")
print(importance_df)

In [1]:
import pandas as pd
        
clean_weather_data = pd.read_csv('datasets/clean_weather_data.csv')
clean_weather_data

,day,tempMin,tempMax,summary,desc,cloudCover,humidity,windSpeed,visibility
0,2015-01-01,7.53,12.23,Possible light rain until evening.,rain,0.80,0.89,14.69,4.43
1,2015-01-02,3.58,7.15,Possible light rain throughout the day.,rain,0.62,0.79,15.04,5.64
2,2015-01-03,-0.61,6.54,Clear throughout the day.,rain,0.31,0.84,4.48,6.20
3,2015-01-04,-0.63,7.59,Mostly cloudy throughout the day.,partly-cloudy-day,0.78,0.85,4.35,6.22
4,2015-01-05,6.51,10.43,Overcast throughout the day.,partly-cloudy-day,0.85,0.91,6.20,5.91
...,...,...,...,...,...,...,...,...,...
1790,2019-11-26,8.55,10.57,Mostly cloudy throughout the day.,rain,0.96,0.85,6.80,10.00
1791,2019-11-27,5.79,9.40,Mostly cloudy throughout the day.,rain,0.87,0.85,6.82,9.03
1792,2019-11-28,2.30,6.67,Mostly cloudy throughout the day.,partly-cloudy-day,0.40,0.79,5.15,10.00
1793,2019-11-29,1.08,6.14,Mostly cloudy throughout the day.,clear-day,0.18,0.76,4.85,9.11


In [2]:
# 选择指定的列
df_selected = clean_weather_data[['tempMin', 'tempMax', 'cloudCover', 'humidity', 'windSpeed', 'visibility']]


In [3]:
df_selected

,tempMin,tempMax,cloudCover,humidity,windSpeed,visibility
0,7.53,12.23,0.80,0.89,14.69,4.43
1,3.58,7.15,0.62,0.79,15.04,5.64
2,-0.61,6.54,0.31,0.84,4.48,6.20
3,-0.63,7.59,0.78,0.85,4.35,6.22
4,6.51,10.43,0.85,0.91,6.20,5.91
...,...,...,...,...,...,...
1790,8.55,10.57,0.96,0.85,6.80,10.00
1791,5.79,9.40,0.87,0.85,6.82,9.03
1792,2.30,6.67,0.40,0.79,5.15,10.00
1793,1.08,6.14,0.18,0.76,4.85,9.11


In [6]:
# 导出为新的 CSV 文件
df_selected.to_csv('useful_data/weather_1795_6.csv', index=False)


In [7]:
import pandas as pd
        
Air_Temperature_C_400_400 = pd.read_csv('useful_data/Air_Temperature_C_400_400.csv')
Air_Temperature_C_400_400

,14.600370500524871,14.600462999819063,14.600558329824013,14.600656486613742,14.60075746625782,14.60086126482139,14.600967878365193,14.601077302945582,14.60118953461453,14.601304569419673,...,14.809819321596047,14.810692351602892,14.811566530151993,14.812441853028433,14.813318316020347,14.814195914918757,14.815074645517784,14.815954503614522,14.816835485009099,14.817717585504766
0,14.600907,14.600999,14.601094,14.601191,14.601292,14.601395,14.601501,14.601609,14.601721,14.601835,...,14.809988,14.810860,14.811733,14.812607,14.813483,14.814359,14.815237,14.816115,14.816995,14.817876
1,14.601441,14.601532,14.601626,14.601723,14.601823,14.601926,14.602031,14.602139,14.602250,14.602364,...,14.810156,14.811027,14.811899,14.812772,14.813646,14.814521,14.815398,14.816275,14.817154,14.818034
2,14.601973,14.602063,14.602157,14.602253,14.602352,14.602454,14.602559,14.602666,14.602777,14.602890,...,14.810324,14.811193,14.812064,14.812936,14.813809,14.814683,14.815558,14.816435,14.817312,14.818191
3,14.602502,14.602592,14.602684,14.602780,14.602879,14.602980,14.603084,14.603191,14.603301,14.603413,...,14.810490,14.811359,14.812228,14.813099,14.813971,14.814844,14.815718,14.816593,14.817470,14.818347
4,14.603028,14.603117,14.603210,14.603305,14.603403,14.603503,14.603607,14.603713,14.603822,14.603934,...,14.810656,14.811523,14.812392,14.813261,14.814132,14.815004,14.815877,14.816751,14.817626,14.818503
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
394,14.641298,14.641528,14.641759,14.641991,14.642225,14.642461,14.642698,14.642936,14.643176,14.643418,...,14.822261,14.822910,14.823560,14.824210,14.824862,14.825513,14.826166,14.826819,14.827473,14.828128
395,14.641077,14.641307,14.641539,14.641773,14.642008,14.642245,14.642483,14.642722,14.642963,14.643206,...,14.822184,14.822833,14.823483,14.824133,14.824784,14.825436,14.826088,14.826742,14.827395,14.828050
396,14.640854,14.641086,14.641319,14.641554,14.641790,14.642028,14.642267,14.642507,14.642749,14.642993,...,14.822107,14.822756,14.823406,14.824056,14.824707,14.825358,14.826011,14.826664,14.827317,14.827971
397,14.640631,14.640863,14.641098,14.641334,14.641571,14.641810,14.642050,14.642291,14.642535,14.642779,...,14.822030,14.822679,14.823328,14.823978,14.824629,14.825280,14.825932,14.826585,14.827239,14.827893


In [8]:
import pandas as pd
        
Dew_Point_C_400_400 = pd.read_csv('useful_data/Dew_Point_C_400_400.csv')
Dew_Point_C_400_400

,8.999907010559944,8.999254827855932,8.99860355370626,8.997953187144024,8.997303727202258,8.99665517291392,8.996007523311826,8.995360777428779,8.994714934297443,8.994069992950458,...,8.805135493317024,8.804766249352053,8.804397556815385,8.804029414886653,8.803661822746108,8.803294779574696,8.802928284553985,8.802562336866233,8.802196935694345,8.801832080221878
0,9.000151,8.999499,8.998848,8.998198,8.997548,8.996900,8.996252,8.995606,8.994960,8.994315,...,8.805388,8.805019,8.804650,8.804282,8.803914,8.803547,8.803180,8.802814,8.802449,8.802084
1,9.000394,8.999742,8.999091,8.998441,8.997792,8.997143,8.996496,8.995849,8.995203,8.994559,...,8.805640,8.805271,8.804902,8.804534,8.804166,8.803799,8.803432,8.803066,8.802701,8.802336
2,9.000636,8.999984,8.999333,8.998683,8.998034,8.997386,8.996738,8.996092,8.995446,8.994802,...,8.805891,8.805522,8.805153,8.804785,8.804417,8.804050,8.803683,8.803317,8.802952,8.802587
3,9.000877,9.000225,8.999574,8.998924,8.998275,8.997627,8.996980,8.996333,8.995688,8.995043,...,8.806142,8.805773,8.805404,8.805036,8.804668,8.804301,8.803934,8.803568,8.803202,8.802837
4,9.001117,9.000465,8.999814,8.999165,8.998516,8.997867,8.997220,8.996574,8.995928,8.995284,...,8.806393,8.806023,8.805654,8.805286,8.804918,8.804551,8.804184,8.803818,8.803452,8.803087
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
394,9.022687,9.022213,9.021740,9.021267,9.020795,9.020324,9.019852,9.019382,9.018911,9.018442,...,8.869372,8.869050,8.868729,8.868408,8.868087,8.867766,8.867446,8.867126,8.866807,8.866488
395,9.022600,9.022128,9.021655,9.021183,9.020712,9.020241,9.019771,9.019301,9.018831,9.018362,...,8.869461,8.869139,8.868818,8.868497,8.868177,8.867856,8.867536,8.867217,8.866897,8.866579
396,9.022514,9.022042,9.021570,9.021099,9.020628,9.020158,9.019688,9.019219,9.018750,9.018282,...,8.869549,8.869228,8.868907,8.868586,8.868266,8.867946,8.867626,8.867307,8.866988,8.866669
397,9.022427,9.021955,9.021485,9.021014,9.020544,9.020075,9.019606,9.019137,9.018669,9.018201,...,8.869638,8.869317,8.868996,8.868675,8.868355,8.868036,8.867716,8.867397,8.867078,8.866760


In [9]:
import pandas as pd
        
Relative_Humidity_400_400 = pd.read_csv('useful_data/Relative_Humidity_400_400.csv')
Relative_Humidity_400_400

,69.43959655478638,69.43653002195505,69.43345515386157,69.43037196296686,69.42728046175176,69.42418066271676,69.42107257838177,69.41795622128642,69.41483160398958,69.41169873906975,...,67.74734725547071,67.7421029288356,67.73685592887216,67.73160627021636,67.72635396749445,67.7210990353233,67.71584148830986,67.7105813410515,67.70531860813584,67.70005330414023
0,69.437913,69.434850,69.431778,69.428698,69.425610,69.422513,69.419408,69.416295,69.413174,69.410044,...,67.747344,67.742104,67.736862,67.731618,67.726370,67.721120,67.715868,67.710613,67.705355,67.700094
1,69.436236,69.433176,69.430108,69.427031,69.423946,69.420852,69.417751,69.414641,69.411523,69.408396,...,67.747342,67.742107,67.736870,67.731630,67.726388,67.721143,67.715895,67.710645,67.705392,67.700137
2,69.434566,69.431509,69.428444,69.425370,69.422288,69.419198,69.416099,69.412993,69.409878,69.406755,...,67.747341,67.742111,67.736879,67.731644,67.726407,67.721166,67.715924,67.710678,67.705430,67.700180
3,69.432902,69.429848,69.426786,69.423715,69.420637,69.417550,69.414455,69.411351,69.408239,69.405119,...,67.747341,67.742116,67.736889,67.731659,67.726427,67.721191,67.715954,67.710713,67.705470,67.700225
4,69.431244,69.428194,69.425135,69.422067,69.418992,69.415908,69.412816,69.409716,69.406607,69.403490,...,67.747343,67.742123,67.736900,67.731675,67.726448,67.721217,67.715985,67.710749,67.705511,67.700270
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
394,69.207684,69.204856,69.202024,69.199188,69.196347,69.193501,69.190652,69.187798,69.184939,69.182076,...,67.838805,67.834784,67.830761,67.826736,67.822709,67.818681,67.814651,67.810619,67.806586,67.802551
395,69.207924,69.205095,69.202262,69.199424,69.196581,69.193735,69.190884,69.188028,69.185168,69.182304,...,67.839221,67.835201,67.831180,67.827157,67.823132,67.819105,67.815077,67.811047,67.807015,67.802982
396,69.208167,69.205336,69.202501,69.199662,69.196818,69.193970,69.191118,69.188261,69.185400,69.182534,...,67.839638,67.835620,67.831600,67.827579,67.823555,67.819531,67.815504,67.811476,67.807446,67.803414
397,69.208412,69.205580,69.202743,69.199903,69.197058,69.194208,69.191354,69.188496,69.185634,69.182767,...,67.840055,67.836039,67.832020,67.828001,67.823979,67.819956,67.815931,67.811905,67.807876,67.803847


In [10]:
import pandas as pd
        
Wind_Speed_kn_400_400 = pd.read_csv('useful_data/Wind_Speed_kn_400_400.csv')
Wind_Speed_kn_400_400

,0.4048684077226471,0.40505876431682497,0.40524862380931903,0.4054379867989836,0.4056268538850435,0.4058152256671045,0.40600310274516427,0.4061904857195765,0.40637737519108574,0.40656377176079206,...,0.44745844985394034,0.4475018435350596,0.4475449799018033,0.44758785953327357,0.4476304830081483,0.44767285090466863,0.44771496380064946,0.4477568222734668,0.447798426900062,0.44783977825694066
0,0.404750,0.404940,0.405130,0.405320,0.405509,0.405697,0.405885,0.406072,0.406259,0.406446,...,0.447367,0.447410,0.447454,0.447497,0.447539,0.447582,0.447624,0.447666,0.447708,0.447749
1,0.404632,0.404823,0.405013,0.405202,0.405391,0.405579,0.405767,0.405955,0.406142,0.406328,...,0.447276,0.447319,0.447362,0.447406,0.447448,0.447491,0.447533,0.447575,0.447617,0.447659
2,0.404515,0.404705,0.404895,0.405085,0.405274,0.405462,0.405650,0.405838,0.406025,0.406211,...,0.447184,0.447228,0.447272,0.447315,0.447358,0.447400,0.447443,0.447485,0.447527,0.447569
3,0.404398,0.404589,0.404779,0.404968,0.405157,0.405345,0.405533,0.405721,0.405908,0.406094,...,0.447093,0.447137,0.447181,0.447224,0.447267,0.447310,0.447352,0.447395,0.447437,0.447479
4,0.404282,0.404472,0.404662,0.404852,0.405041,0.405229,0.405417,0.405605,0.405792,0.405978,...,0.447003,0.447047,0.447090,0.447134,0.447177,0.447220,0.447262,0.447305,0.447347,0.447389
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
394,0.393879,0.394005,0.394131,0.394256,0.394381,0.394506,0.394631,0.394756,0.394880,0.395004,...,0.426328,0.426375,0.426421,0.426468,0.426515,0.426561,0.426607,0.426653,0.426699,0.426745
395,0.393920,0.394046,0.394171,0.394297,0.394422,0.394546,0.394671,0.394795,0.394919,0.395042,...,0.426305,0.426352,0.426399,0.426445,0.426492,0.426538,0.426584,0.426630,0.426676,0.426722
396,0.393962,0.394087,0.394212,0.394337,0.394462,0.394586,0.394711,0.394834,0.394958,0.395081,...,0.426283,0.426330,0.426376,0.426423,0.426469,0.426516,0.426562,0.426608,0.426653,0.426699
397,0.394004,0.394129,0.394254,0.394378,0.394503,0.394627,0.394751,0.394874,0.394997,0.395120,...,0.426260,0.426307,0.426354,0.426400,0.426447,0.426493,0.426539,0.426585,0.426631,0.426676


In [ ]:
Air_Temperature_C_400_400

In [11]:
import rasterio
from rasterio.transform import from_origin
import numpy as np

In [12]:
# 定义地理空间范围和分辨率（根据需要修改）
transform = from_origin(258000, 666000, 5, 5)  # 左上角坐标(0, 0)，分辨率为1x1


In [14]:
Air_Temperature_C_400_400_np = Air_Temperature_C_400_400.to_numpy()

In [15]:
# 将数据导出为 TIFF 文件
with rasterio.open(
        'Air.tif',
        'w',
        driver='GTiff',
        height=Air_Temperature_C_400_400_np.shape[0],
        width=Air_Temperature_C_400_400_np.shape[1],
        count=1,
        dtype=Air_Temperature_C_400_400_np.dtype,
        crs='+proj=latlong',
        transform=transform,
) as dst:
    dst.write(Air_Temperature_C_400_400_np, 1)

print("TIFF 文件已成功导出为 output.tif")

TIFF 文件已成功导出为 output.tif


In [34]:
C_overlap_data

array([[[1.79e+308, 1.79e+308, 1.79e+308, ..., 1.79e+308, 1.79e+308,
         1.79e+308],
        [1.79e+308, 1.79e+308, 1.79e+308, ..., 1.79e+308, 1.79e+308,
         1.79e+308],
        [1.79e+308, 1.79e+308, 1.79e+308, ..., 1.79e+308, 1.79e+308,
         1.79e+308],
        ...,
        [1.79e+308, 1.79e+308, 1.79e+308, ..., 1.79e+308, 1.79e+308,
         1.79e+308],
        [1.79e+308, 1.79e+308, 1.79e+308, ..., 1.79e+308, 1.79e+308,
         1.79e+308],
        [1.79e+308, 1.79e+308, 1.79e+308, ..., 1.79e+308, 1.79e+308,
         1.79e+308]]])

In [35]:
BV_tif = C_overlap_data[0, :, :]

In [36]:
BV_tif

array([[1.79e+308, 1.79e+308, 1.79e+308, ..., 1.79e+308, 1.79e+308,
        1.79e+308],
       [1.79e+308, 1.79e+308, 1.79e+308, ..., 1.79e+308, 1.79e+308,
        1.79e+308],
       [1.79e+308, 1.79e+308, 1.79e+308, ..., 1.79e+308, 1.79e+308,
        1.79e+308],
       ...,
       [1.79e+308, 1.79e+308, 1.79e+308, ..., 1.79e+308, 1.79e+308,
        1.79e+308],
       [1.79e+308, 1.79e+308, 1.79e+308, ..., 1.79e+308, 1.79e+308,
        1.79e+308],
       [1.79e+308, 1.79e+308, 1.79e+308, ..., 1.79e+308, 1.79e+308,
        1.79e+308]])

In [39]:
np.savetxt('useful_data/BV_tif_400_400.csv', BV_tif, delimiter=',')